In [5]:
import logging
import os
from datetime import datetime
print(datetime.now())

2023-12-07 10:28:01.239649


In [6]:
LOG_FILE=f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [7]:
os.getcwd()

'c:\\Users\\DEELL\\Desktop\\fsdsmendtoend\\notebook'

In [8]:
os.path.join(os.getcwd(),"logs")

'c:\\Users\\DEELL\\Desktop\\fsdsmendtoend\\notebook\\logs'

In [1]:
from src.DimondPricePridiction.logger import logging

In [2]:
logging.info(' i have just tested the things')

In [22]:
os.path.join(os.getcwd(),"data\\gemstone.csv")

'c:\\Users\\DEELL\\Desktop\\fsdsmendtoend\\notebook\\data\\gemstone.csv'

In [118]:
from pathlib import Path
import pandas as pd
import os
data= Path(os.path.join(os.getcwd(),"data\gemstone.csv"))
#data=  Path("notebook\\data","gemstone.csv")
#os.path.join("notebook\\data","gemstone.csv")
data = pd.read_csv(data)
data


,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [145]:
import os,sys
import pandas as pd
import numpy as np
from src.DimondPricePridiction.logger import logging
from src.DimondPricePridiction.exception import customexception
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
from pathlib import Path

class DataIngestionConfig:
    raw_data_path:str= os.path.join("artifacts","raw.csv")
    train_data_path:str= os.path.join("artifacts","train.csv")
    test_data_path:str= os.path.join("artifacts","test.csv") 



class DataIngestion:


    
    def __init__(self):
        self.ingestion_config=DataIngestionConfig()

    def initiate_data_ingestion(self):
        logging.info("data ingestion started")

        try:
            data= str(Path(os.path.join(os.getcwd(),"data\gemstone.csv")))
            #data= pd.read_csv(Path(os.path.join("notebook/data","gemstone.csv")))
            #data= pd.read_csv(os.path.join(os.getcwd(),"data\\gemstone.csv"))
            logging.info(" i have read dataset as a df")
            
            os.makedirs(os.path.join(self.ingestion_config.raw_data_path),exist_ok= True)
            data.to_csv(self.ingestion_config.raw_data_path, index=False)
            logging.info(" i have saved the raw dataset in artifact folder")


            logging.info(" i have performed train test split")
            
            train_data, test_data= train_test_split(data,test_size=0.25)
            logging.info(" train test split completed")

            train_data.to_csv(self.ingestion_config.train_data_path, index=False)
            test_data.to_csv(self.ingestion_config.test_data_path, index=False)
            
            logging.info("data ingestion part completed")
            
        except Exception as e:
            logging.info("exception during data ingestion steps")
            raise customexception(e,sys)


In [2]:
import os
import sys
import pandas as pd
import numpy as np

from src.DimondPricePridiction.exception import customexception
from src.DimondPricePridiction.logger import logging
from dataclasses import dataclass

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from src.DimondPricePridiction.utils.utils import save_object


In [3]:
@dataclass
class DataTransformationConfig:
    preprocessor_obj_file_path=os.path.join('artifacts','preprocessor.pkl')

In [ ]:



class DataTransformation:
    
    def __init__(self):
        self.data_transformation_config= DataTransformationConfig()

    def get_data_transformation(self):
        try:
            
            logging.info("Data transformation initiated")

            # Define which column should be ordinal encoded and which should be scaled
            categorical_cols= ['cut','clarity','color']
            numerical_cols= ['carat','depth','table','x','y','z']

            # Define the custom ranking for each ordinal variable
            cut_categories= ['Fair','Good','Very Good','Premium','Ideal']
            color_categories= ['D','E','F','G','H','I','J']
            clarity_categories= ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

            logging.info("Pipeline initiated")

            #Numerical Pipeline
            num_pipeline= Pipeline(
                steps=   [
                ('imputer',SimpleImputer(strategy='median')),
                ('scaler',StandardScaler())
            ])

            #Categorical Pipeline
            cat_pipeline= Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
                ('scaler',StandardScaler())
                ]
            )

            preprocessor= ColumnTransformer([
                ('num_pipeline',num_pipeline,numerical_cols),
                ('cat_pipeline',cat_pipeline,categorical_cols)
            ])
            return preprocessor

        except Exception as e:
            logging.info("error initiate in data transformation")
            raise customexception(e,sys)
        
    def initialize_data_transformation(self, train_path, test_path):
        try:
            train_df= pd.read_csv(train_path)
            test_df= pd.read_csv(test_path)

            logging.info("read train and test data complete")
            logging.info(f'Train data head : \n{train_df.head().to_string()}')
            logging.info(f'Test data head : \n{test_df.head().to_string()}')
            
            preprocessing_obj= self.get_data_transformation()
            
            target_column_name= 'price'
            drop_columns= [target_column_name,'id']

            input_feature_train_df= train_df.drop(columns=drop_columns, axis= 1)
            target_feature_train_df= train_df[target_column_name]

            input_feature_test_df= test_df.drop(columns=drop_columns, axis= 1)
            target_feature_test_df= test_df[target_column_name]

            input_feature_train_arr= preprocessing_obj.fit_transform(input_feature_train_df)
            input_feature_test_arr= preprocessing_obj.transform(input_feature_test_df)

            
            logging.info("Applying preprocessing on train and test datasets")

            save_object(
                file_path= self.data_transformation_config.preprocessor_obj_file_path,
                obj= preprocessing_obj
            )
            
            return(
                input_feature_train_arr,
                input_feature_test_arr
            )
        except Exception as e:
            logging.info("Exception initiate in data ingestion")
            raise customexception(e,sys)
        

In [4]:
def __init__(self):
        self.data_transformation_config= DataTransformationConfig()

In [5]:
def get_data_transformation(self):
        try:
            
            logging.info("Data transformation initiated")

            # Define which column should be ordinal encoded and which should be scaled
            categorical_cols= ['cut','clarity','color']
            numerical_cols= ['carat','depth','table','x','y','z']

            # Define the custom ranking for each ordinal variable
            cut_categories= ['Fair','Good','Very Good','Premium','Ideal']
            color_categories= ['D','E','F','G','H','I','J']
            clarity_categories= ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

            logging.info("Pipeline initiated")

            #Numerical Pipeline
            num_pipeline= Pipeline(
                steps=   [
                ('imputer',SimpleImputer(strategy='median')),
                ('scaler',StandardScaler())
            ])

            #Categorical Pipeline
            cat_pipeline= Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
                ('scaler',StandardScaler())
                ]
            )

            preprocessor= ColumnTransformer([
                ('num_pipeline',num_pipeline,numerical_cols),
                ('cat_pipeline',cat_pipeline,categorical_cols)
            ])
            return preprocessor

        except Exception as e:
            logging.info("error initiate in data transformation")
            raise customexception(e,sys)
    

In [6]:
def initialize_data_transformation(self, train_path, test_path):
        try:
            train_df= pd.read_csv(train_path)
            test_df= pd.read_csv(test_path)

            logging.info("read train and test data complete")
            logging.info(f'Train data head : \n{train_df.head().to_string()}')
            logging.info(f'Test data head : \n{test_df.head().to_string()}')
            
            preprocessing_obj= self.get_data_transformation()
            
            target_column_name= 'price'
            drop_columns= [target_column_name,'id']

            input_feature_train_df= train_df.drop(columns=drop_columns, axis= 1)
            target_feature_train_df= train_df[target_column_name]

            input_feature_test_df= test_df.drop(columns=drop_columns, axis= 1)
            target_feature_test_df= test_df[target_column_name]

            input_feature_train_arr= preprocessing_obj.fit_transform(input_feature_train_df)
            input_feature_test_arr= preprocessing_obj.transform(input_feature_test_df)

            
            logging.info("Applying preprocessing on train and test datasets")

            save_object(
                file_path= self.data_transformation_config.preprocessor_obj_file_path,
                obj= preprocessing_obj
            )
            
            return(
                input_feature_train_arr,
                input_feature_test_arr
            )
        except Exception as e:
            logging.info("Exception initiate in data ingestion")
            raise customexception(e,sys)
        